In [59]:
# from Feat_eng import Y_train, X_train, Y_test, X_test, c1_train, c2_train, c3_train, c1_test, c2_test, c3_test, Y_c1train, Y_c2train, Y_c3train, Y_c1test, Y_c2test ,Y_c3test

In [60]:
import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score, RandomizedSearchCV
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import statsmodels.api as sm
import pylab as py
import scipy.stats as stats 
from scipy.stats import norm, skew, probplot
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [61]:
from Pred_ZRI_feats import zip_test, zip_train, future_feats, future_targets, Y_train, X_train, Y_test, X_test, c1_train, c2_train, c3_train, c1_test, c2_test, c3_test, Y_c1train, Y_c2train, Y_c3train, Y_c1test, Y_c2test ,Y_c3test, target

In [62]:
target

,RegionName,year,value,geo_id,nonfamily_households,family_households,median_year_structure_built,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,...,log_value,future_value,male_ratio,white_ratio,black_ratio,asian_ratio,hispanic_ratio,employed_ratio,civil_labor_ratio,Cluster
0,01104,2014,946.100000,01104,3346.0,5526.0,1949.0,158.0,1597.0,523.0,...,6.852348,972.583333,0.477361,0.329594,0.117433,0.010904,0.531703,0.395109,0.469239,1
1,01104,2015,972.583333,01104,3377.0,5407.0,1948.0,146.0,1791.0,466.0,...,6.879956,1058.666667,0.478263,0.319353,0.101512,0.019630,0.549670,0.398254,0.461912,1
2,01104,2016,1058.666667,01104,3432.0,5434.0,1949.0,172.0,1898.0,516.0,...,6.964766,1129.166667,0.467110,0.315878,0.079972,0.017987,0.571669,0.392658,0.449424,1
3,01104,2017,1129.166667,01104,3287.0,5702.0,1946.0,107.0,2032.0,577.0,...,7.029235,1125.083333,0.460292,0.292194,0.078781,0.012735,0.596573,0.388238,0.438968,1
4,01104,2018,1125.083333,01104,3478.0,5599.0,1946.0,75.0,1963.0,502.0,...,7.025612,1145.583333,0.463101,0.275423,0.086447,0.013905,0.604750,0.391816,0.436003,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6042,98499,2014,979.000000,98499,5610.0,6870.0,1973.0,265.0,2355.0,952.0,...,6.886532,995.833333,0.492245,0.467639,0.103251,0.086748,0.211496,0.397328,0.469392,1
6043,98499,2015,995.833333,98499,5736.0,6891.0,1973.0,346.0,2355.0,816.0,...,6.903580,1036.166667,0.505450,0.463689,0.108836,0.087294,0.221283,0.396388,0.464463,1
6044,98499,2016,1036.166667,98499,5670.0,6965.0,1973.0,310.0,2352.0,995.0,...,6.943283,1154.833333,0.502808,0.439341,0.120046,0.093177,0.210382,0.403395,0.459354,1
6045,98499,2017,1154.833333,98499,5149.0,7244.0,1973.0,327.0,2367.0,1006.0,...,7.051711,1257.416667,0.510771,0.425098,0.124402,0.108897,0.206512,0.412859,0.463662,1


In [63]:
future_feats.shape

(2574, 137)

In [64]:
future_targets.shape

(2574, 3)

In [65]:
future_feats

,year,nonfamily_households,family_households,median_year_structure_built,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,...,est,log_value,male_ratio,white_ratio,black_ratio,asian_ratio,hispanic_ratio,employed_ratio,civil_labor_ratio,Cluster
0,2019,3514.005750,5619.596240,1945.251283,64.221758,2069.917185,500.311260,181.584777,552.064005,465.324405,...,504.060577,7.069693,0.459636,0.263420,0.080712,0.015700,0.624568,0.391020,0.428081,1
1,2020,3550.011500,5640.192481,1944.502565,53.443516,2176.834370,498.622521,161.169554,608.128010,440.648811,...,500.121154,7.113774,0.456172,0.251416,0.074977,0.017495,0.644386,0.390223,0.420160,1
2,2021,3586.017251,5660.788721,1943.753848,42.665274,2283.751555,496.933781,140.754331,664.192015,415.973216,...,496.181731,7.157854,0.452707,0.239413,0.069242,0.019291,0.664204,0.389427,0.412239,1
3,2019,3370.873725,6619.614626,0.000000,311.662957,2014.843245,729.487100,354.197443,677.049498,760.454862,...,248.656208,7.084782,0.478647,0.295612,0.161785,0.047261,0.467141,0.421469,0.459697,1
4,2020,3409.747451,6693.229251,0.000000,344.325913,2032.686491,782.974201,379.394886,696.098996,808.909725,...,246.312417,7.092137,0.479053,0.284688,0.162744,0.046371,0.481499,0.424769,0.457455,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569,2020,3839.149961,7492.102714,1972.000254,207.124040,647.776627,379.302655,415.846214,846.355676,769.799947,...,330.828118,7.349357,0.490387,0.574766,0.123798,0.083781,0.115791,0.453177,0.460586,1
2570,2021,3756.724942,7609.154072,1972.000381,210.686060,575.664940,377.453983,433.269321,922.533514,858.199920,...,341.242176,7.422146,0.491915,0.569195,0.130759,0.084051,0.116907,0.463085,0.463251,1
2571,2019,5366.584888,7717.252494,1971.750127,533.233881,2199.355425,973.789775,615.147082,906.323242,874.294928,...,1268.491165,7.200888,0.517350,0.426119,0.136041,0.093197,0.214187,0.423036,0.452032,1
2572,2020,5325.169777,7895.504988,1971.500253,608.467762,2169.710849,984.579549,635.294164,988.646483,905.589855,...,1274.982330,7.264961,0.522501,0.418256,0.143377,0.095051,0.214856,0.428323,0.448631,1


In [66]:
with open("Pickle_lasso_all.pkl" , 'rb') as file:  
     lasso_all = pickle.load(file)

In [67]:
with open("Pickle_lasso_all_PCA.pkl" , 'rb') as file:  
     lasso_all_PCA = pickle.load(file)

In [68]:
with open("Pickle_lasso_c1.pkl" , 'rb') as file:  
     lasso_c1 = pickle.load(file)

In [69]:
with open("Pickle_lasso_c1_PCA.pkl" , 'rb') as file:  
     lasso_c1_PCA = pickle.load(file)

In [70]:
with open("Pickle_lasso_c2.pkl" , 'rb') as file:  
     lasso_c2 = pickle.load(file)

In [71]:
with open("Pickle_lasso_c2_PCA.pkl" , 'rb') as file:  
     lasso_c2_PCA = pickle.load(file)

In [72]:
with open("Pickle_lasso_c3.pkl" , 'rb') as file:  
     lasso_c2 = pickle.load(file)

In [73]:
with open("Pickle_lasso_c3_PCA.pkl" , 'rb') as file:  
     lasso_c2_PCA = pickle.load(file)

In [74]:
with open("rf_all.pkl" , 'rb') as file:  
     rf_all = pickle.load(file)

In [75]:
list(future_feats.isnull().sum())

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [76]:
list(zip(future_feats.columns, list(future_feats.max())))

[('year', 2021.0),
 ('nonfamily_households', 24961.959410419437),
 ('family_households', 39191.43550797895),
 ('median_year_structure_built', 2017.7311023079146),
 ('rent_burden_not_computed', 2848.4269941944117),
 ('rent_over_50_percent', 12073.40810342431),
 ('rent_40_to_50_percent', 3186.730045744383),
 ('rent_35_to_40_percent', 2460.3496850851934),
 ('rent_30_to_35_percent', 3759.892474697628),
 ('rent_25_to_30_percent', 3899.5252270303718),
 ('rent_20_to_25_percent', 4483.325450814269),
 ('rent_15_to_20_percent', 4376.319911928115),
 ('rent_10_to_15_percent', 3883.8962934748374),
 ('rent_under_10_percent', 4032.3474486861496),
 ('total_pop', 148814.68682023592),
 ('median_age', 62.158737450659274),
 ('commuters_by_public_transportation', 36490.498611471776),
 ('households', 46486.47985255736),
 ('median_income', 179693.74810481275),
 ('income_per_capita', 157709.9979141879),
 ('housing_units', 50209.35217695162),
 ('vacant_housing_units', 12429.59346359319),
 ('vacant_housing_unit

In [77]:
future_feats_2020 = future_feats[future_feats['year']==2020]

In [78]:
future_targets_2021 = future_targets[future_targets['year']==2021]

In [79]:
print(rf_all.best_params_)
print(rf_all.best_score_)
Y_pred = rf_all.predict(future_feats_2020)
pred_unlog = np.exp(Y_pred)
Y_unlog = future_targets_2021[['ZRI']]
train_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The RMSE is:', train_RMSE)

{'n_estimators': 19, 'min_samples_split': 2, 'min_samples_leaf': 7, 'max_samples': 0.7}
0.9787581366670949
The RMSE is: 155.6823800441864


In [80]:
Y_pred_test = np.exp(rf_all.predict(X_test))

In [81]:
Y_actual = np.exp(Y_test)

In [82]:
zip_test

,RegionName,City
0,01104,Springfield
1,01108,Springfield
2,01109,Springfield
3,01602,Worcester
4,01603,Worcester
...,...,...
920,98125,Seattle
921,98201,Everett
922,98203,Everett
923,98498,Lakewood


In [83]:
diff_df = pd.DataFrame()

In [84]:
diff_df['Zip'] = zip_test['RegionName']
diff_df['City'] = zip_test['City']
diff_df['Pred'] = Y_pred_test
diff_df['Act'] = np.exp(Y_test)

In [85]:
diff_df['Model_Error'] = diff_df['Pred'] - diff_df['Act']

In [86]:
diff_df

,Zip,City,Pred,Act,Model_Error
0,01104,Springfield,1150.714148,1145.583333,5.130815
1,01108,Springfield,1192.499645,1201.500000,-9.000355
2,01109,Springfield,1171.390547,1196.727273,-25.336726
3,01602,Worcester,1462.432603,1401.333333,61.099270
4,01603,Worcester,1324.319494,1325.583333,-1.263839
...,...,...,...,...,...
920,98125,Seattle,1761.159632,1645.200000,115.959632
921,98201,Everett,1454.265146,1450.916667,3.348480
922,98203,Everett,1519.896025,1510.916667,8.979359
923,98498,Lakewood,1373.151584,1409.000000,-35.848416


In [87]:
comparison = pd.DataFrame()

In [88]:
comparison['Zip'] = future_targets_2021['RegionName']

In [89]:
comparison['Year'] = future_targets_2021['year']

In [90]:
comparison['NoCOVID_ZRI'] = pred_unlog

In [91]:
comparison['COVID_ZRI'] = future_targets_2021[['ZRI']]

In [92]:
comparison['Difference'] = comparison['NoCOVID_ZRI'] - comparison['COVID_ZRI']

In [93]:
comparison = comparison[comparison['Year']==2021]

In [94]:
test = comparison.merge(diff_df, how = 'left', on = 'Zip')

In [95]:
test['Pct'] = (test['Difference'] - test['Model_Error'])/test['Model_Error']

In [96]:
test['Diff_Greater_Than_Error'] = (abs(test['Difference'])>abs(test['Model_Error']))

In [97]:
test['Adjusted_Diff'] = test['Difference'] - test['Model_Error']

In [98]:
test['Pct_Diff'] = test['Adjusted_Diff']/test['NoCOVID_ZRI']

In [99]:
test_true = test[test['Diff_Greater_Than_Error'] == True]

In [100]:
test_true.sort_values(by = 'Pct_Diff').head(20)

,Zip,Year,NoCOVID_ZRI,COVID_ZRI,Difference,City,Pred,Act,Model_Error,Pct,Diff_Greater_Than_Error,Adjusted_Diff,Pct_Diff
832,97232,2021,1601.529341,1691.735657,-90.206316,Portland,1696.388918,1644.416667,51.972251,-2.735663,True,-142.178568,-0.088777
359,32209,2021,797.198911,834.765626,-37.566715,Jacksonville,844.080543,811.416667,32.663877,-2.150100,True,-70.230591,-0.088097
314,27603,2021,1224.936865,1317.861477,-92.924612,Raleigh,1265.614813,1281.000000,-15.385187,5.039875,True,-77.539425,-0.063301
581,77006,2021,1391.992825,1446.887077,-54.894252,Houston,1425.627350,1406.416667,19.210683,-3.857486,True,-74.104935,-0.053237
448,47374,2021,666.685362,690.879935,-24.194572,Richmond,681.265020,671.555556,9.709465,-3.491854,True,-33.904037,-0.050855
276,20009,2021,2206.814309,2450.543355,-243.729045,Washington,2241.659876,2382.000000,-140.340124,0.736703,True,-103.388922,-0.046850
72,10013,2021,4096.851514,4582.633915,-485.782401,New York,4158.944133,4454.454545,-295.510413,0.643876,True,-190.271988,-0.046443
628,78731,2021,1457.496885,1610.875458,-153.378573,Austin,1471.537226,1565.818182,-94.280955,0.626825,True,-59.097617,-0.040547
247,15217,2021,1245.111908,1277.139111,-32.027204,Pittsburgh,1258.270995,1241.416667,16.854329,-2.900236,True,-48.881532,-0.039259
278,20011,2021,1923.188363,2014.248998,-91.060635,Washington,1937.855418,1957.909091,-20.053673,3.540846,True,-71.006962,-0.036921


In [101]:
test_true.sort_values(by = 'Pct_Diff', ascending = False).head(20)

,Zip,Year,NoCOVID_ZRI,COVID_ZRI,Difference,City,Pred,Act,Model_Error,Pct,Diff_Greater_Than_Error,Adjusted_Diff,Pct_Diff
543,75034,2021,2219.062987,1685.237225,533.825762,Frisco,1535.308416,1638.100000,-102.791584,-6.193283,True,636.617346,0.286886
803,94602,2021,3401.134242,2738.257583,662.876659,Oakland,2631.470659,2661.666667,-30.196008,-22.952460,True,693.072667,0.203777
824,95816,2021,1884.005793,1624.779515,259.226277,Sacramento,1455.905324,1579.333333,-123.428009,-3.100222,True,382.654287,0.203107
825,95831,2021,1941.569517,1618.263937,323.305580,Sacramento,1562.530555,1573.000000,-10.469445,-31.880871,True,333.775025,0.171910
808,94608,2021,3474.075462,2942.983917,531.091545,Oakland,2798.474457,2860.666667,-62.192210,-9.539519,True,593.283755,0.170775
805,94605,2021,2976.312706,2539.532440,436.780266,Oakland,2407.046460,2468.500000,-61.453540,-8.107487,True,498.233806,0.167400
472,55406,2021,1837.010712,1485.271316,351.739396,Minneapolis,1509.300211,1443.727273,65.572939,4.364094,True,286.166458,0.155778
852,98121,2021,2810.177054,2379.931942,430.245112,Seattle,2307.625440,2313.363636,-5.738197,-75.979149,True,435.983309,0.155144
366,32811,2021,1620.354649,1373.329625,247.025024,Orlando,1332.872035,1334.916667,-2.044632,-121.816380,True,249.069656,0.153713
830,97214,2021,1928.611360,1641.084049,287.527311,Portland,1588.715838,1595.181818,-6.465980,-45.467705,True,293.993292,0.152438


In [102]:
test_true.sort_values(by = 'Pct_Diff', key = abs, ascending = False).head(20)

,Zip,Year,NoCOVID_ZRI,COVID_ZRI,Difference,City,Pred,Act,Model_Error,Pct,Diff_Greater_Than_Error,Adjusted_Diff,Pct_Diff
543,75034,2021,2219.062987,1685.237225,533.825762,Frisco,1535.308416,1638.100000,-102.791584,-6.193283,True,636.617346,0.286886
803,94602,2021,3401.134242,2738.257583,662.876659,Oakland,2631.470659,2661.666667,-30.196008,-22.952460,True,693.072667,0.203777
824,95816,2021,1884.005793,1624.779515,259.226277,Sacramento,1455.905324,1579.333333,-123.428009,-3.100222,True,382.654287,0.203107
825,95831,2021,1941.569517,1618.263937,323.305580,Sacramento,1562.530555,1573.000000,-10.469445,-31.880871,True,333.775025,0.171910
808,94608,2021,3474.075462,2942.983917,531.091545,Oakland,2798.474457,2860.666667,-62.192210,-9.539519,True,593.283755,0.170775
805,94605,2021,2976.312706,2539.532440,436.780266,Oakland,2407.046460,2468.500000,-61.453540,-8.107487,True,498.233806,0.167400
472,55406,2021,1837.010712,1485.271316,351.739396,Minneapolis,1509.300211,1443.727273,65.572939,4.364094,True,286.166458,0.155778
852,98121,2021,2810.177054,2379.931942,430.245112,Seattle,2307.625440,2313.363636,-5.738197,-75.979149,True,435.983309,0.155144
366,32811,2021,1620.354649,1373.329625,247.025024,Orlando,1332.872035,1334.916667,-2.044632,-121.816380,True,249.069656,0.153713
830,97214,2021,1928.611360,1641.084049,287.527311,Portland,1588.715838,1595.181818,-6.465980,-45.467705,True,293.993292,0.152438


In [103]:
test_true.sort_values(by = 'Pct_Diff', key = abs).head(20)

,Zip,Year,NoCOVID_ZRI,COVID_ZRI,Difference,City,Pred,Act,Model_Error,Pct,Diff_Greater_Than_Error,Adjusted_Diff,Pct_Diff
627,78729,2021,1306.770677,1291.970625,14.800052,Austin,1270.404639,1255.833333,14.571305,0.015698,True,0.228747,0.000175
525,63104,2021,865.087728,879.345899,-14.258172,Saint Louis,841.052571,854.750000,-13.697429,0.040938,True,-0.560743,-0.000648
53,06513,2021,1304.206095,1275.853142,28.352953,New Haven,1267.163163,1240.166667,26.996496,0.050246,True,1.356457,0.001040
162,11232,2021,2224.998180,2125.278819,99.719361,New York,2163.034525,2065.833333,97.201192,0.025907,True,2.518169,0.001132
450,53204,2021,759.364073,729.659057,29.705016,Milwaukee,738.024586,709.250000,28.774586,0.032335,True,0.930430,0.001225
57,06605,2021,1493.719111,1448.258777,45.460333,Bridgeport,1450.618472,1407.750000,42.868472,0.060461,True,2.591862,0.001735
90,10075,2021,3149.651620,3154.319355,-4.667735,New York,3066.957856,3066.090909,0.866947,-6.384107,True,-5.534682,-0.001757
678,85716,2021,809.852235,776.639807,33.212427,Tucson,785.570112,754.916667,30.653446,0.083481,True,2.558982,0.003160
521,60654,2021,2238.110195,2225.670166,12.440029,Chicago,2168.524640,2163.416667,5.107974,1.435414,True,7.332055,0.003276
495,60618,2021,1760.823892,1690.745850,70.078042,Chicago,1707.603985,1643.454545,64.149439,0.092419,True,5.928603,0.003367


In [235]:
top_zip = test_true.groupby('Zip')[['Pct_Diff']].mean().sort_values(by = 'Pct_Diff', ascending = False).head(30)
top_zip = top_zip.reset_index()

In [236]:
top_zip2 = top_zip.merge(test_true[['Zip', 'City']], on='Zip', how='left')
top_zip2

,Zip,Pct_Diff,City
0,75034,0.286886,Frisco
1,94602,0.203777,Oakland
2,95816,0.203107,Sacramento
3,95831,0.171910,Sacramento
4,94608,0.170775,Oakland
5,94605,0.167400,Oakland
6,55406,0.155778,Minneapolis
7,98121,0.155144,Seattle
8,32811,0.153713,Orlando
9,97214,0.152438,Portland


In [245]:
re_gp = top_zip2.groupby('City').mean('Pct_Diff').sort_values(by = 'Pct_Diff', ascending = False)
re_gp = re_gp.reset_index()

In [246]:
re_gp

,City,Pct_Diff
0,Frisco,0.286886
1,Sacramento,0.187508
2,Oakland,0.159924
3,Portland,0.151590
4,Orlando,0.138450
5,Minneapolis,0.137578
6,Seattle,0.136751
7,Duluth,0.136275
8,San Jose,0.135720
9,Olathe,0.133391


In [237]:
# grouping by city - bottom percentages
bot_zip = test_true.groupby('Zip')[['Pct_Diff']].mean().sort_values(by = 'Pct_Diff', ascending = True).head(30)
bot_zip.reset_index()

,Zip,Pct_Diff
0,97232,-0.088777
1,32209,-0.088097
2,27603,-0.063301
3,77006,-0.053237
4,47374,-0.050855
5,20009,-0.046850
6,10013,-0.046443
7,78731,-0.040547
8,15217,-0.039259
9,20011,-0.036921


In [238]:
bot_zip2 = bot_zip.merge(test_true[['Zip', 'City']], on='Zip', how='left')
bot_zip2

,Zip,Pct_Diff,City
0,97232,-0.088777,Portland
1,32209,-0.088097,Jacksonville
2,27603,-0.063301,Raleigh
3,77006,-0.053237,Houston
4,47374,-0.050855,Richmond
5,20009,-0.046850,Washington
6,10013,-0.046443,New York
7,78731,-0.040547,Austin
8,15217,-0.039259,Pittsburgh
9,20011,-0.036921,Washington


In [247]:
re_gp2 = bot_zip2.groupby('City').mean('Pct_Diff').sort_values(by = 'Pct_Diff', ascending = False)
re_gp2 = re_gp2.reset_index()

In [248]:
re_gp2

,City,Pct_Diff
0,Springfield,-0.023224
1,Buffalo,-0.023774
2,Bradenton,-0.024905
3,Philadelphia,-0.026183
4,Alexandria,-0.026541
5,Chicago,-0.026857
6,Rochester,-0.029002
7,Killeen,-0.030011
8,Cleveland,-0.031644
9,Pittsburgh,-0.032895


In [146]:
top_cit_list = top_cit['City'].tolist()
top_cit_list

['Frisco',
 'Sacramento',
 'Oakland',
 'Duluth',
 'Minneapolis',
 'Orlando',
 'Olathe',
 'Bellevue',
 'Seattle',
 'Mount Pleasant',
 'Lawrenceville',
 'Aurora',
 'Everett',
 'San Jose',
 'Huntington Beach',
 'Las Vegas',
 'Redmond',
 'Henderson',
 'Plano',
 'Pasadena']

In [ ]:
bot_cit['City']

In [105]:
with open("rf_all_PCA.pkl" , 'rb') as file:  
     rf_all_PCA = pickle.load(file)

In [106]:
with open("rf_cluster1.pkl" , 'rb') as file:  
     rf_c1 = pickle.load(file)

In [107]:
with open("rf_cluster2.pkl" , 'rb') as file:  
     rf_c1 = pickle.load(file)

In [108]:
with open("rf_cluster3.pkl" , 'rb') as file:  
     rf_c1 = pickle.load(file)

### Map of Cities

In [250]:
zip_lat_long = pd.read_csv('../data/2013_zip_us.csv')
zip_lat_long.columns = ['zip', 'lat', 'long']
zip_lat_long['zip'] = zip_lat_long['zip'].astype(str).str.zfill(5)
zip_lat_long

,zip,lat,long
0,00601,18.180555,-66.749961
1,00602,18.361945,-67.175597
2,00603,18.455183,-67.119887
3,00606,18.158345,-66.932911
4,00610,18.295366,-67.125135
...,...,...,...
33139,99923,56.002315,-130.041026
33140,99925,55.550204,-132.945933
33141,99926,55.138352,-131.470424
33142,99927,56.239062,-133.457924


In [270]:
top_ = top_zip2.merge(zip_lat_long, how = 'left', left_on = 'Zip', right_on = 'zip')
top_

,Zip,Pct_Diff,City,zip,lat,long
0,75034,0.286886,Frisco,75034,33.146638,-96.855822
1,94602,0.203777,Oakland,94602,37.804320,-122.207545
2,95816,0.203107,Sacramento,95816,38.571577,-121.466465
3,95831,0.171910,Sacramento,95831,38.493966,-121.529188
4,94608,0.170775,Oakland,94608,37.836844,-122.289143
5,94605,0.167400,Oakland,94605,37.761408,-122.150198
6,55406,0.155778,Minneapolis,55406,44.938689,-93.221042
7,98121,0.155144,Seattle,98121,47.615192,-122.350581
8,32811,0.153713,Orlando,32811,28.521568,-81.447421
9,97214,0.152438,Portland,97214,45.514672,-122.643014


In [281]:
top_['color'] = '#00ae53'

In [272]:
print(top_['Pct_Diff'].max())
print(top_['Pct_Diff'].min())

0.28688565828963053
0.12033908154769524


In [282]:
# top_zip map in folium
import folium
import branca.colormap as cm

colormap = cm.StepColormap(
    colors=['#00ae53'])

location = [42.0267,-93.6465]
map_1 = folium.Map(location=location, tiles="cartodbpositron",
                  zoom_start=3)
top_.apply(lambda row: folium.CircleMarker(
    location=[row['lat'],row['long']],
    radius = 3,
    color=row["color"], fill=True).add_to(map_1), axis=1)

bot_.apply(lambda row: folium.CircleMarker(
    location=[row['lat'],row['long']],
    radius = 3,
    color=row["color"], fill=True).add_to(map_1), axis=1)
map_1

In [274]:
bot_ = bot_zip2.merge(zip_lat_long, how = 'left', left_on = 'Zip', right_on = 'zip')
bot_

,Zip,Pct_Diff,City,zip,lat,long
0,97232,-0.088777,Portland,97232,45.528929,-122.643927
1,32209,-0.088097,Jacksonville,32209,30.361015,-81.696254
2,27603,-0.063301,Raleigh,27603,35.664418,-78.652751
3,77006,-0.053237,Houston,77006,29.740970,-95.391301
4,47374,-0.050855,Richmond,47374,39.832079,-84.889699
5,20009,-0.046850,Washington,20009,38.919144,-77.037381
6,10013,-0.046443,New York,10013,40.720103,-74.004903
7,78731,-0.040547,Austin,78731,30.348236,-97.768136
8,15217,-0.039259,Pittsburgh,15217,40.430821,-79.920089
9,20011,-0.036921,Washington,20011,38.952514,-77.022991


In [277]:
bot_['color'] = '#D22B2B'

In [278]:
# bot_zip map in folium

location = [42.0267,-93.6465]
map_2 = folium.Map(location=location, tiles="cartodbpositron",
                  zoom_start=3)
bot_.apply(lambda row: folium.CircleMarker(
    location=[row['lat'],row['long']],
    radius = 3,
    color=row["color"], fill=True).add_to(map_1), axis=1)
map_1

In [283]:
test_true

,Zip,Year,NoCOVID_ZRI,COVID_ZRI,Difference,City,Pred,Act,Model_Error,Pct,Diff_Greater_Than_Error,Adjusted_Diff,Pct_Diff
0,01104,2021,1281.080478,1178.548121,102.532356,Springfield,1150.714148,1145.583333,5.130815,18.983641,True,97.401542,0.076031
2,01109,2021,1178.458504,1231.163756,-52.705252,Springfield,1171.390547,1196.727273,-25.336726,1.080192,True,-27.368526,-0.023224
3,01602,2021,1523.836936,1441.657468,82.179469,Worcester,1462.432603,1401.333333,61.099270,0.345016,True,21.080199,0.013834
4,01603,2021,1484.401260,1363.727720,120.673540,Worcester,1324.319494,1325.583333,-1.263839,-96.481736,True,121.937379,0.082146
5,01604,2021,1505.402712,1403.904523,101.498189,Worcester,1356.708486,1364.636364,-7.927877,-13.802694,True,109.426067,0.072689
...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,98122,2021,2251.816340,1982.279018,269.537322,Seattle,1936.922015,1926.833333,10.088682,25.716802,True,259.448640,0.115217
854,98201,2021,1607.202655,1492.667589,114.535067,Everett,1454.265146,1450.916667,3.348480,33.205095,True,111.186587,0.069180
855,98203,2021,1769.736453,1554.394121,215.342332,Everett,1519.896025,1510.916667,8.979359,22.981928,True,206.362973,0.116607
856,98498,2021,1586.426231,1449.544747,136.881484,Lakewood,1373.151584,1409.000000,-35.848416,-4.818341,True,172.729899,0.108880


In [284]:
all_ = test_true.merge(zip_lat_long, how = 'left', left_on = 'Zip', right_on = 'zip')
all_

,Zip,Year,NoCOVID_ZRI,COVID_ZRI,Difference,City,Pred,Act,Model_Error,Pct,Diff_Greater_Than_Error,Adjusted_Diff,Pct_Diff,zip,lat,long
0,01104,2021,1281.080478,1178.548121,102.532356,Springfield,1150.714148,1145.583333,5.130815,18.983641,True,97.401542,0.076031,01104,42.134009,-72.565378
1,01109,2021,1178.458504,1231.163756,-52.705252,Springfield,1171.390547,1196.727273,-25.336726,1.080192,True,-27.368526,-0.023224,01109,42.119674,-72.549726
2,01602,2021,1523.836936,1441.657468,82.179469,Worcester,1462.432603,1401.333333,61.099270,0.345016,True,21.080199,0.013834,01602,42.269189,-71.850728
3,01603,2021,1484.401260,1363.727720,120.673540,Worcester,1324.319494,1325.583333,-1.263839,-96.481736,True,121.937379,0.082146,01603,42.243826,-71.843569
4,01604,2021,1505.402712,1403.904523,101.498189,Worcester,1356.708486,1364.636364,-7.927877,-13.802694,True,109.426067,0.072689,01604,42.253254,-71.767957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,98122,2021,2251.816340,1982.279018,269.537322,Seattle,1936.922015,1926.833333,10.088682,25.716802,True,259.448640,0.115217,98122,47.611046,-122.293207
683,98201,2021,1607.202655,1492.667589,114.535067,Everett,1454.265146,1450.916667,3.348480,33.205095,True,111.186587,0.069180,98201,48.006311,-122.210044
684,98203,2021,1769.736453,1554.394121,215.342332,Everett,1519.896025,1510.916667,8.979359,22.981928,True,206.362973,0.116607,98203,47.945519,-122.231096
685,98498,2021,1586.426231,1449.544747,136.881484,Lakewood,1373.151584,1409.000000,-35.848416,-4.818341,True,172.729899,0.108880,98498,47.159194,-122.553122


In [303]:
# map of all the zip codes
colormap = cm.StepColormap(
    colors=['#f16913', '#d94801', '#8c2d04', '#41ae76', '#238b45', '#005824'],
    vmin=min(all_['Pct_Diff']),
    vmax=max(all_['Pct_Diff']),
    index=np.linspace(min(all_['Pct_Diff']), max(all_['Pct_Diff']), 7))

map_2 = folium.Map(
    location=[42.0267,-93.6465],
    tiles='cartodbpositron',
    zoom_start=3,
)
all_.apply(lambda row:folium.CircleMarker(location=[row["lat"], row["long"]], radius = 4, fill=True, 
                                     color=colormap(row['Pct_Diff'])).add_to(map_2), axis=1)

map_2


In [288]:
np.arange(min(all_['Pct_Diff']), max(all_['Pct_Diff']), .05)

array([-0.08877675, -0.03877675,  0.01122325,  0.06122325,  0.11122325,
        0.16122325,  0.21122325,  0.26122325])

In [290]:
np.linspace(min(all_['Pct_Diff']), max(all_['Pct_Diff']), 7)

array([-0.08877675, -0.02616635,  0.03644405,  0.09905445,  0.16166486,
        0.22427526,  0.28688566])